Functions

In [3]:
from copy import deepcopy
import numpy as np
def gauss(A, B):
  A = np.linalg.inv(A)
  AB = np.dot(A,B)
  return(AB)

def transposed(matriz):
    trans = []
    for i in range(len(matriz[0])):
        aux = []
        for j in range(len(matriz)):
            aux.append(matriz[j][i])
        trans.append(aux)

    return trans


def mult(a, b):
    result = 0
    if len(a) != len(b):
        print('Gauss error')
        exit()

    for i in range(len(a)):
        result += a[i] * b[i]

    return result


def column(A, index):
    col = []
    for i in range(len(A)):
        col.append(A[i][index])
    return col


def print_matrix(mat, nome):
    print('\nMatrix {}:'.format(nome))
    for i in range(len(mat)):
        for j in range(len(mat[0])):
            print(mat[i][j], end='\t')
        print()
    print('\n')

from tabulate import tabulate
from prettytable import PrettyTable
from termcolor import colored


#add the identity matrix to matrix A
def add_identity(A, quant_rest):
    for i in range(quant_rest):
        for j in range(quant_rest):
            if i == j:
                A[i].append(1)
            else:
                A[i].append(0)


#deletes artificial variables from the list of non-basic ones and deletes columns related to artificial variables
def fimFase1(A, nVarArtificial, Non_basic_vars):
    for i in range(nVarArtificial):
        Non_basic_vars.remove(len(A[0]) - 1 - i)

    for i in range(len(A)):
        for j in range(nVarArtificial):
            A[i].pop(-1)


def standard_form(problem_type, objective_function, restrictions, operators, b):
    fase1 = True
    A = []
    basic_vars = []
    Non_basic_vars = []
    nVarArtificial = 0

    if problem_type == 'max':  #turns to min if max
        objective_function = [(x * -1) for x in objective_function]

    for i in range(len(b)): #check for negative constraint and multiply the line by -19
        if b[i] < 0:
            b[i] *= -1
            for j in range(len(restrictions[0])):
                restrictions[i][j] *= -1

            if operators[i] == '<=':
                operators[i] = '>='
            elif operators[i] == '>=':
                operators[i] = '<='

    #creation of matrix A
    for i in range(len(restrictions)):
        lin = []
        for j in range(len(restrictions[0])):
            lin.append(restrictions[i][j])
        A.append(lin)

    #adds columns of slack and excess variables
    for i in range(len(operators)):
        if operators[i] == '<=':
            objective_function.append(0)  #adds the cost of the slack variable to the objective function
            for j in range(len(A)):
                if i == j:
                    A[j].append(1)
                else:
                    A[j].append(0)
        elif operators[i] == '>=':
            objective_function.append(0)  #adds the cost of the excess variable to the objective function
            for j in range(len(A)):
                if i == j:
                    A[j].append(-1)
                else:
                    A[j].append(0)

    #checks the need for phase 1
#     if '<=' in operators:
#         fase1 = True

    #selects the first basic variables
    for i in range(len(restrictions[0])):
        Non_basic_vars.append(i)

    # selects basic and non-basic variables
    if fase1:
        add_identity(A, len(restrictions))
        nVarArtificial = len(restrictions)

        for i in range(len(restrictions)):
            basic_vars.append(len(A[0]) - len(restrictions) + i)
            objective_function.append(0)

        for i in range(len(Non_basic_vars), len(A[0]) - len(restrictions)):
            Non_basic_vars.append(i)
    else:
        for i in range(len(restrictions)):
            basic_vars.append(i + len(restrictions[0]))

    print_matrix(A, 'A')
    print('Objective function: {}'.format(objective_function))
    print('Vector b: {}'.format(b))
    #ic('Vector b: {}'.format(b))
    print('Index of Basic variables: {}'.format(basic_vars))
    print('Index of Non-basic variables: {}'.format(Non_basic_vars))
    print('Fase 1: {}'.format(fase1))
    print('Number of artificial variables: {}'.format(nVarArtificial))

    return A, objective_function, b, basic_vars, Non_basic_vars, fase1, nVarArtificial


def update_B(A, BS):
    B = []

    for i in range(len(A)):
        line = []
        for j in range(len(BS)):
            line.append(A[i][BS[j]])

        B.append(line)

    return B

#n=len(basic_vars) + len(Non_basic_vars)
def step1(B, b, basic_vars, n):
    XB = gauss(B, b)

    X = [0] * n
    for i in range(len(basic_vars)):
        X[basic_vars[i]] = XB[i]

    print('XB :', end='')
    for i in XB:
        print('{:.2f}'.format(i), end=' ')
    print()

    print('X:', end='')
    for i in X:
        print('{:.2f}'.format(i), end=' ')
    print()
    return XB, X


def step2_1(B, basic_vars, objective_function):
    costs = []
    for i in basic_vars:
        costs.append(objective_function[i]) # takes the costs of basic variables in the objective function
    lbda = gauss(transposed(B), costs) # performs the gauss between the transposed B matrix and the costs of the basic variables

    print('lambda:', end='')
    for i in lbda:
        print('{:.2f}'.format(i), end=' ')
    print()
    #print('vetor lbda: {}'.format(lbda))
    return lbda


def step2_2(objective_function, lbda, A, Non_basic_vars):
    vet_cnk = []
    for cn in range(len(Non_basic_vars)):
        cnk = objective_function[Non_basic_vars[cn]] - mult(lbda, column(A, Non_basic_vars[cn]))
        vet_cnk.append(cnk)

    print('vet_cnk: ', end='')
    for i in vet_cnk:
        print('{:.2f}'.format(i), end=' ')
    print()
    return vet_cnk

 #step 2.3 and 3
def step3(vet_cnk, step1):
    finish= False
    menor = min(vet_cnk)
    k = vet_cnk.index(menor)
    if menor < 0:
        print('\033[35msolution is not optimal\033[m')
        print('Entering: SN{}'.format(k + 1))
    elif step1 is True:
        finish = True
        print('\033[31mInfeasible problem\033[m')
    else:
        print('\033[35mOptimal solution found\033[m')
        finish = True
    return finish, k


def step4(B, A, Non_basic_vars, k):
    y = gauss(B, column(A, Non_basic_vars[k]))

    print('y: ', end='')
    for i in y:
        print('{:.2f}'.format(i), end=' ')
    print()
    #print("y=",y)
    return y


def step5(XB, y):
    if max(y) <= 0:
        print('Unlimited optimal solution')
        return None, True
    #aux mokhafafe komakie
    aux = []
    for i in range(len(y)):
        if y[i] > 0:
            aux.append(XB[i] / y[i])
        else:
            aux.append(-1)

    menor = [aux[0], 0]
    for i in range(1, len(aux)):
        if (menor[0] > aux[i] > 0) or menor[0] <= 0:
            menor[0] = aux[i]
            menor[1] = i
    leaver = menor[1]
    print('Leaving: B{}'.format(leaver + 1))
    return leaver, False


def step5_fase1(XB, y):
    if max(y) <= 0:
        print('Infeasible Original Problem')
        return None, True

    aux = float('inf')
    leaver = 0

    for i in range(len(XB)):
        if y[i] > 0 and XB[i]/y[i] < aux:
            aux = XB[i]/y[i]
            leaver = i

    print('Leaving: B{}'.format(leaver + 1))
    return leaver, False


def step6(BS, N, entrant, leaver):
    aux = BS[leaver]
    BS[leaver] = N[entrant]
    N[entrant] = aux
    return BS, N


def step6_fase1(BS, N, entrant, leaver, nVarArtificial):
    finish = True
    aux = BS[leaver]
    BS[leaver] = N[entrant]
    N[entrant] = aux

    totalVars = len(BS) + len(N)
    for i in range(len(BS)):
        for j in range(nVarArtificial):
            if BS[i] == totalVars - 1 - j:
                finish = False

    return BS, N, finish


def printInfo(basic_vars, Non_basic_vars, B, itr,B_inv,objective_function,CB):
    print('\n')
    print('\033[34mIteration: {}\033[m'.format(itr))
    print_matrix(B, 'B')
    print_matrix(B_inv, 'B_inverse')
    print('Index of basic variables: {}'.format(basic_vars))
    print('Index of non-basic variables: {}'.format(Non_basic_vars))
    print('\n')
    print('CB :',CB)
        

def final(restrictions, objective_function, problem_type, X):
    print('\n\n')
    optimal_solution = 0
    for i in range(len(restrictions[0])):
        optimal_solution += objective_function[i] * X[i]

    for i in range(len(restrictions[0])):
        print('X{} = {}'.format(i + 1, X[i]))
    #if tipo_prob == 'max':
    if problem_type == 'max':
        optimal_solution *= -1
    print('Optimal solution = {}'.format(optimal_solution))

Main solver

In [4]:
'''*******************************************Primal****************************************************'''
def primal():
    problem_type = 'min'
    objective_function0 = [15, 10, 9, 7]
    

    restrictions = [[1, 1, 1, 1],
                  [0, 0, 1, 0],
                  [2, 3, 4, 5],
                  [3, 4, 5, 6]]

                  
    
    operators = ['=',
                 '>=',
                 '<=',
                 '<=']
                
    b0= [1000, 400, 3300, 4000]

###################################### Dual entrants #####################################
    d_operators=[]
    if problem_type=='max':
        d_problem_type='min'
        rest=np.array(restrictions)
        dual_rest=rest.T
        d_restrictions=dual_rest.tolist()
        d_b=objective_function0
        d_objective_function0=b0
        
        for i in range(len(d_b)):
            d_operators.append('>=')
    else:
        d_problem_type='max'
        rest=np.array(restrictions)
        dual_rest=rest.T
        d_restrictions=dual_rest.tolist()
        d_b=objective_function0
        d_objective_function0=b0
        for i in range(len(d_b)):
            d_operators.append('<=')
    
   
 ########################################################################################
    obj=[]
    for i in range(len(objective_function0)):
        obj.append(objective_function0[i])
    A, objective_function, b, basic_vars, Non_basic_vars, fase1, nVarArtificial = standard_form(problem_type, objective_function0, restrictions, operators, b0)

    problem = False

    if fase1:
        itr = 1
        objective_function_fase1 = [0] * len(objective_function)
        for i in range(len(objective_function_fase1) - 1, len(objective_function_fase1) - 1 - nVarArtificial, -1): # assigns 1 for all costs of artificial variables
            objective_function_fase1[i] = 1

        print('\n\n\033[33mFase I\033[m')
        print('Objective function of fase I: {}'.format(objective_function_fase1))
  

        while True:
            B = update_B(A, basic_vars)
            B_inv=np.linalg.inv(B)
            CB=[]
            for i in basic_vars:
                CB.append(objective_function_fase1[i])
            
            printInfo(basic_vars, Non_basic_vars, B, itr,B_inv,objective_function,CB)
            Xb, X = step1(B, b, basic_vars, len(basic_vars) + len(Non_basic_vars))
            lbda = step2_1(B, basic_vars, objective_function_fase1)
            vet_cnk = step2_2(objective_function_fase1, lbda, A, Non_basic_vars)
            ###
            Binvdotrest=np.dot(B_inv, restrictions)
            CBdot=np.dot(CB,Binvdotrest)          
            CBXb=np.dot(CB,Xb)
            CBBinv=np.dot(CB,B_inv)
            BinvA=np.dot(B_inv,restrictions)
            
            ###
            finish, k = step3(vet_cnk, True)

            if finish:
                break

            y = step4(B, A, Non_basic_vars, k)
            leaver, problem = step5_fase1(Xb, y)

            if problem:
                break

            basic_vars, Non_basic_vars, finish = step6_fase1(basic_vars, Non_basic_vars, k, leaver, nVarArtificial)

            if finish:
                break

            itr += 1

        fimFase1(A, nVarArtificial, Non_basic_vars)

    itr = 1
    X = []

    print('\n\n\033[33mFase II\033[m')

    while True:
        

        B = update_B(A, basic_vars)
        print('xxxxxxxxxxxxxxxxxxxx',id(False))
       
        B_inv=np.linalg.inv(B)
        mB_inv=np.array(B_inv)
        CB=[]
        for i in basic_vars:
            CB.append(objective_function[i])
        mCB=np.array(CB)
        printInfo(basic_vars, Non_basic_vars, B, itr,B_inv,objective_function,CB)
        Xb, X = step1(B, b, basic_vars, len(basic_vars) + len(Non_basic_vars))

        row_titles=['z']
        for i in basic_vars:
            row_titles.append('x'+str(i+1))
        mrow_titles=np.array(row_titles)
        reshaped_row_titles=mrow_titles.reshape(len(row_titles),1)
       
                
        lbda = step2_1(B, basic_vars, objective_function)
        vet_cnk = step2_2(objective_function, lbda, A, Non_basic_vars)


        len_mXb=len(Xb)
        mXb=np.array(Xb)
        Binvdotrest=np.dot(B_inv, restrictions)
        mBinvdotrest=np.array(Binvdotrest)
        CBBinvA=np.dot(CB,Binvdotrest)
        mCBBinvA=np.array(CBBinvA)
        if problem_type == 'min':
            CBBinvA_c=CBBinvA-obj
        else:
            CBBinvA_c=CBBinvA+obj
        mCBBinvA_c=np.array(CBBinvA_c)        
        CBXb=np.dot(CB,Xb)
        mCBXb=np.array(CBXb)
        CBBinv=np.dot(CB,B_inv)
        mCBBinv=np.array(CBBinv)
        BinvA=np.dot(B_inv,restrictions)
        mBinvA=np.array(BinvA)
        BinvABinv=np.concatenate((mBinvA,mB_inv),1)
        mXb=mXb.reshape((len_mXb,1))
        BinvABinvXb=np.concatenate((BinvABinv,mXb),1)
        CBBinvA_cCBBinv=np.concatenate((mCBBinvA_c,mCBBinv),None)
        row0=np.concatenate((CBBinvA_cCBBinv, mCBXb),None)
       
        len_row0=len(row0.tolist())
        
        table=np.concatenate((row0.reshape((1,len_row0)),BinvABinvXb),0)
        table=np.concatenate((reshaped_row_titles,table),1)

        p,u=np.shape(table)
        x = PrettyTable()
        column_num=['_']
        for j in range(u-1):
            column_num.append('X'+str(j+1))
        column_num[j+1]='RHS'
        for i in range(p):
            x.add_row(table[i,:])

        x.field_names = column_num
        print(x)

                              
        finish, k = step3(vet_cnk, False)

        if finish:
            break

        y = step4(B, A, Non_basic_vars, k)
        leaver, problem = step5(Xb, y)

        if problem:
            break

        basic_vars, Non_basic_vars = step6(basic_vars, Non_basic_vars, k, leaver)
        
        itr += 1

    if not problem:
        print('\033[33mFinal Primal Table\033[m')
        final(restrictions, objective_function, problem_type, X)

    return(d_problem_type,d_restrictions,d_objective_function0,d_operators,obj,b0)


d_problem_type,d_restrictions,d_objective_function0,d_operators,d_b,b0 =primal()
print('d_problem_type :',d_problem_type,'/d_restrictions :',d_restrictions,'/d_objective_function0 :',d_objective_function0,'/d_operators :',d_operators,'/d_b :',d_b)



Matrix A:
1	1	1	1	0	0	0	1	0	0	0	
0	0	1	0	-1	0	0	0	1	0	0	
2	3	4	5	0	1	0	0	0	1	0	
3	4	5	6	0	0	1	0	0	0	1	


Objective function: [15, 10, 9, 7, 0, 0, 0, 0, 0, 0, 0]
Vector b: [1000, 400, 3300, 4000]
Index of Basic variables: [7, 8, 9, 10]
Index of Non-basic variables: [0, 1, 2, 3, 4, 5, 6]
Fase 1: True
Number of artificial variables: 4


Fase I
Objective function of fase I: [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]


Iteration: 1

Matrix B:
1	0	0	0	
0	1	0	0	
0	0	1	0	
0	0	0	1	



Matrix B_inverse:
1.0	0.0	0.0	0.0	
0.0	1.0	0.0	0.0	
0.0	0.0	1.0	0.0	
0.0	0.0	0.0	1.0	


Index of basic variables: [7, 8, 9, 10]
Index of non-basic variables: [0, 1, 2, 3, 4, 5, 6]


CB : [1, 1, 1, 1]
XB :1000.00 400.00 3300.00 4000.00 
X:0.00 0.00 0.00 0.00 0.00 0.00 0.00 1000.00 400.00 3300.00 4000.00 
lambda:1.00 1.00 1.00 1.00 
vet_cnk: -6.00 -8.00 -11.00 -12.00 1.00 -1.00 -1.00 
solution is not optimal
Entering: SN4
y: 1.00 0.00 5.00 6.00 
Leaving: B3


Iteration: 2

Matrix B:
1	0	1	0	
0	1	0	0	
0	0	5	0	
0	0	6	1	



Ma

Dual

In [5]:
'''*******************************************Dual**************************************************'''
def dual():
    problem_type = 'max'
    objective_function0 = [12, 10]
    

    restrictions = [[2, 2],
                  [4, 2]
                   
                        ]

                  
    
    operators = ['<=',
                 '<='
                ]
                
    b0= [5, 3]
#******************************************************************************************************#
    obj=[]
    for i in range(len(objective_function0)):
        obj.append(objective_function0[i])
    A, objective_function, b, basic_vars, Non_basic_vars, fase1, nVarArtificial = standard_form(problem_type, objective_function0, restrictions, operators, b0)

    problem = False

    if fase1:
        itr = 1
        objective_function_fase1 = [0] * len(objective_function)
        #print('ooooooooooooo:',objective_function_fase1)
        for i in range(len(objective_function_fase1) - 1, len(objective_function_fase1) - 1 - nVarArtificial, -1): # assigns 1 for all costs of artificial variables
            objective_function_fase1[i] = 1

        print('\n\n\033[33mFase I\033[m')
        print('Objective function of fase I: {}'.format(objective_function_fase1))
        #print('Vector b: {}'.format(b))

        while True:
            B = update_B(A, basic_vars)
            B_inv=np.linalg.inv(B)
            #print('ooooooooooo',B)
            #print('B_inverse',np.linalg.inv(B))
            CB=[]
            for i in basic_vars:
                #print('basic_vars:',objective_function[i-1])
                CB.append(objective_function_fase1[i])
            
            printInfo(basic_vars, Non_basic_vars, B, itr,B_inv,objective_function,CB)
            Xb, X = step1(B, b, basic_vars, len(basic_vars) + len(Non_basic_vars))
            lbda = step2_1(B, basic_vars, objective_function_fase1)
            vet_cnk = step2_2(objective_function_fase1, lbda, A, Non_basic_vars)
            ###
            Binvdotrest=np.dot(B_inv, restrictions)
            #print('Binvdotrest',Binvdotrest)
            CBdot=np.dot(CB,Binvdotrest)
            print('CB*B^-1*A:',CBdot)
           
            CBXb=np.dot(CB,Xb)
            print('CB*Xb :',CBXb)
            CBBinv=np.dot(CB,B_inv)
            print('CB*B^-1 :',CBBinv)
            BinvA=np.dot(B_inv,restrictions)
            print('B^-1*A :', BinvA)
            #print_matrix(CBdot_c,'CB*B^-1*A')
            
            ###
            finish, k = step3(vet_cnk, True)

            if finish:
                break

            y = step4(B, A, Non_basic_vars, k)
            leaver, problem = step5_fase1(Xb, y)

            if problem:
                break

            basic_vars, Non_basic_vars, finish = step6_fase1(basic_vars, Non_basic_vars, k, leaver, nVarArtificial)

            if finish:
                break

            itr += 1

        fimFase1(A, nVarArtificial, Non_basic_vars)

    itr = 1
    X = []

    print('\n\n\033[33mFase II\033[m')
    while True:
        

        B = update_B(A, basic_vars)
        #print('ooooooooooooooo',B)
        B_inv=np.linalg.inv(B)
        mB_inv=np.array(B_inv)
        CB=[]
        for i in basic_vars:
            #print('basic_vars:',objective_function[i-1])
            CB.append(objective_function[i])
        mCB=np.array(CB)
        printInfo(basic_vars, Non_basic_vars, B, itr,B_inv,objective_function,CB)
        Xb, X = step1(B, b, basic_vars, len(basic_vars) + len(Non_basic_vars))
        for i in Xb:
            if i==0:
                print(colored('we have degeneracy!','red'))
        row_titles=['z']
        for i in basic_vars:
            row_titles.append('x'+str(i+1))
        #print('ooooooooooooooooooooooo',row_titles,np.shape(row_titles))
        #d=np.shape(row_titles)
        #print('oooooooooooooooo',d)
        #len_row_titles =len(row_titles.tolist())
        mrow_titles=np.array(row_titles)
        reshaped_row_titles=mrow_titles.reshape(len(row_titles),1)
        #print('ooooooooooooooooooooooo',reshaped_row_titles,np.shape(reshaped_row_titles))
        
                
#         Xbnew=[]
#         for j in range Xb :
#             Xbnew.append(j)
#         print('Xbnew:',Xbnew)
        lbda = step2_1(B, basic_vars, objective_function)
        vet_cnk = step2_2(objective_function, lbda, A, Non_basic_vars)
        
#         print('typeXb:',type(Xb))
#         print('typebasic_vars:',type(basic_vars))
#         print('typeB:',type(B))
        
       
        #print(np.shape(XbT))
        #print_matrix(Xb,'XbT')
        
        #pp= np.dot(CB, (np.dot(B_inv, restrictions))
        len_mXb=len(Xb)
        mXb=np.array(Xb)
        Binvdotrest=np.dot(B_inv, restrictions)
        mBinvdotrest=np.array(Binvdotrest)
        #print('Binvdotrest',Binvdotrest)
        CBBinvA=np.dot(CB,Binvdotrest)
        mCBBinvA=np.array(CBBinvA)
        print('CB*B^-1*A:',CBBinvA)
        if problem_type == 'min':
            CBBinvA_c=CBBinvA-obj
        else:
            CBBinvA_c=CBBinvA+obj
        mCBBinvA_c=np.array(CBBinvA_c)
        print('(CB*B^-1*A)-c :',mCBBinvA_c)
        
        CBXb=np.dot(CB,Xb)
        mCBXb=np.array(CBXb)
        print('CB*Xb :',CBXb)
        CBBinv=np.dot(CB,B_inv)
        mCBBinv=np.array(CBBinv)
        print('CB*B^-1 :',CBBinv)
        BinvA=np.dot(B_inv,restrictions)
        mBinvA=np.array(BinvA)
        print('B^-1*A :', BinvA)
        BinvABinv=np.concatenate((mBinvA,mB_inv),1)
        #print('B^-1*A/B^-1 :',BinvABinv)
        mXb=mXb.reshape((len_mXb,1))
        BinvABinvXb=np.concatenate((BinvABinv,mXb),1)
        print('B^-1*A/B^-1/B^-1*b :',BinvABinvXb)
#         print('shape1',np.shape(BinvABinv),'shape2',np.shape(mXb.T))
#         print('type of mXb:',type(mXb))
        CBBinvA_cCBBinv=np.concatenate((mCBBinvA_c,mCBBinv),None)
    #   print('CB*B^-1*A/CB*B^-1:',CBBinvACBBinv)
        #print('shape1',np.shape(mCBBinvA),'shape2',np.shape(mCBBinv))
        row0=np.concatenate((CBBinvA_cCBBinv, mCBXb),None)
       
        print('CB*B^-1*A-c/CB*B^-1/CB*B^-1*b:', row0)
        len_row0=len(row0.tolist())
        
        table=np.concatenate((row0.reshape((1,len_row0)),BinvABinvXb),0)
        table=np.concatenate((reshaped_row_titles,table),1)

        p,u=np.shape(table)
        #print(p,u)
        x = PrettyTable()
        column_num=['_']
        for j in range(u-1):
            column_num.append('X'+str(j+1))
        column_num[j+1]='RHS'
        for i in range(p):
            x.add_row(table[i,:])
#         print('oooooooooo',column_num)
#         x.add_row(table[1,:])
#         x.add_row(table[2,:])
        x.field_names = column_num
        print(x)
#         print('ooooooooooooooooooooooooooo',table[0,:])
#         print('XXXXXXXXXXXXX',table.shape,'vvvvvv',type(table))

        
        
        
                              
        finish, k = step3(vet_cnk, False)

        if finish:
            break

        y = step4(B, A, Non_basic_vars, k)
        leaver, problem = step5(Xb, y)

        if problem:
            break

        basic_vars, Non_basic_vars = step6(basic_vars, Non_basic_vars, k, leaver)
        
        itr += 1

    if not problem:
        print('\033[33mFinal Dual Table\033[m')
        final(restrictions, objective_function, problem_type, X)
        
    return
dual()






Matrix A:
2	2	1	0	1	0	
4	2	0	1	0	1	


Objective function: [-12, -10, 0, 0, 0, 0]
Vector b: [5, 3]
Index of Basic variables: [4, 5]
Index of Non-basic variables: [0, 1, 2, 3]
Fase 1: True
Number of artificial variables: 2


Fase I
Objective function of fase I: [0, 0, 0, 0, 1, 1]


Iteration: 1

Matrix B:
1	0	
0	1	



Matrix B_inverse:
1.0	0.0	
0.0	1.0	


Index of basic variables: [4, 5]
Index of non-basic variables: [0, 1, 2, 3]


CB : [1, 1]
XB :5.00 3.00 
X:0.00 0.00 0.00 0.00 5.00 3.00 
lambda:1.00 1.00 
vet_cnk: -6.00 -4.00 -1.00 -1.00 
CB*B^-1*A: [6. 4.]
CB*Xb : 8.0
CB*B^-1 : [1. 1.]
B^-1*A : [[2. 2.]
 [4. 2.]]
solution is not optimal
Entering: SN1
y: 2.00 4.00 
Leaving: B2


Iteration: 2

Matrix B:
1	2	
0	4	



Matrix B_inverse:
1.0	-0.5	
0.0	0.25	


Index of basic variables: [4, 0]
Index of non-basic variables: [5, 1, 2, 3]


CB : [1, 0]
XB :3.50 0.75 
X:0.75 0.00 0.00 0.00 3.50 0.00 
lambda:1.00 -0.50 
vet_cnk: 1.50 -1.00 -1.00 0.50 
CB*B^-1*A: [0. 1.]
CB*Xb : 3.5
CB*B^-1 : [ 1